# Japan Gas Demand Forecasting Models

## Overview

This notebook implements and compares multiple forecasting approaches for Japanese natural gas demand:

### Model Categories

1. **Traditional Time Series Models**
   - ARIMA/SARIMA with automatic parameter selection
   - Exponential Smoothing (Holt-Winters)
   - State Space models

2. **Machine Learning Models**
   - Random Forest with lag features
   - XGBoost for gradient boosting
   - Support Vector Regression

3. **Advanced Methods**
   - Prophet for handling holidays and multiple seasonalities
   - Ensemble methods combining multiple models

### Evaluation Framework

- Time series cross-validation with walk-forward approach
- Multiple accuracy metrics: MAE, RMSE, MAPE, Directional Accuracy
- Statistical significance testing
- Residual analysis and diagnostic checking


In [ ]:
# Import required libraries for forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import joblib

# Time series modeling
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from prophet import Prophet

# Evaluation metrics
from sklearn.metrics import mean_absolute_percentage_error

# Plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Import our custom modules
import sys
sys.path.append('../src')
from forecasting_utils import (
    calculate_forecast_metrics, time_series_cv_split, 
    ForecastEvaluator, plot_forecast_results
)

# Set styling and suppress warnings
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')
np.random.seed(42)

print("🚀 Japan Gas Demand Forecasting Models")
print("=" * 50)
print(f"📅 Model development started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Forecasting libraries imported successfully")
print("✅ Custom utilities loaded")


In [ ]:
# Load the processed dataset
print("📊 LOADING DATASET FOR MODELING")
print("=" * 40)

try:
    # Try to load from the processed data file
    gas_data = pd.read_csv('../data/processed/japan_gas_demand_processed.csv', index_col=0, parse_dates=True)
    print("✅ Loaded processed dataset from file")
except FileNotFoundError:
    print("⚠️ Processed data file not found, generating fresh dataset...")
    # Generate fresh data if file doesn't exist
    from data_processing import JapanGasDataCollector
    collector = JapanGasDataCollector()
    gas_data = collector.generate_synthetic_data('2018-01-01', '2024-08-31')
    gas_data = collector.add_calendar_features(gas_data)
    gas_data = collector.create_lagged_features(gas_data, 'total_gas_demand_mcm', max_lag=12)
    gas_data = collector.clean_and_validate_data(gas_data)
    print("✅ Generated fresh synthetic dataset")

# Prepare data for modeling
target_col = 'total_gas_demand_mcm'
y = gas_data[target_col].dropna()

print(f"📈 Dataset Overview:")
print(f"   • Period: {gas_data.index.min().strftime('%Y-%m')} to {gas_data.index.max().strftime('%Y-%m')}")
print(f"   • Total observations: {len(gas_data):,}")
print(f"   • Target variable: {target_col}")
print(f"   • Valid target observations: {len(y):,}")

# Display basic statistics
print(f"\n📊 Target Variable Statistics:")
print(f"   • Mean: {y.mean():.2f} million m³/month")
print(f"   • Std Dev: {y.std():.2f} million m³/month")
print(f"   • Min: {y.min():.2f} million m³/month")
print(f"   • Max: {y.max():.2f} million m³/month")
print(f"   • Coefficient of Variation: {(y.std()/y.mean()*100):.1f}%")

# Check stationarity
print(f"\n🔍 Stationarity Check:")
stationarity_result = adfuller(y)
print(f"   • ADF Statistic: {stationarity_result[0]:.4f}")
print(f"   • P-value: {stationarity_result[1]:.4f}")
print(f"   • Critical Values:")
for key, value in stationarity_result[4].items():
    print(f"     {key}: {value:.4f}")
print(f"   • Stationary: {'Yes' if stationarity_result[1] < 0.05 else 'No'}")

# Prepare feature matrix for ML models
feature_cols = [
    'avg_temperature_celsius', 'heating_degree_days', 'cooling_degree_days',
    'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos',
    'is_winter', 'is_spring', 'is_summer', 'is_autumn',
    'gdp_growth_rate_pct', 'industrial_production_index'
]

# Add lag features
lag_cols = [col for col in gas_data.columns if 'lag_' in col or 'ma_' in col or 'std_' in col]
feature_cols.extend(lag_cols)

# Filter available features
available_features = [col for col in feature_cols if col in gas_data.columns]
print(f"\n🔧 Feature Engineering:")
print(f"   • Available features: {len(available_features)}")
print(f"   • Feature categories:")
print(f"     - Weather: {len([c for c in available_features if 'temp' in c or 'hdd' in c or 'cdd' in c])}")
print(f"     - Seasonal: {len([c for c in available_features if 'sin' in c or 'cos' in c or 'is_' in c])}")
print(f"     - Lag/Rolling: {len([c for c in available_features if 'lag_' in c or 'ma_' in c or 'std_' in c])}")
print(f"     - Economic: {len([c for c in available_features if 'gdp' in c or 'industrial' in c])}")

# Create feature matrix
X = gas_data[available_features].dropna()
y_aligned = gas_data.loc[X.index, target_col]

print(f"   • Final dataset shape: X={X.shape}, y={y_aligned.shape}")
print(f"   • Date range for modeling: {X.index.min().strftime('%Y-%m')} to {X.index.max().strftime('%Y-%m')}")


In [ ]:
# === TRADITIONAL TIME SERIES MODELS ===
print("📈 TRADITIONAL TIME SERIES FORECASTING")
print("=" * 50)

# Prepare data for time series models
y_ts = gas_data[target_col].dropna()
print(f"Time series data: {len(y_ts)} observations")

# 1. ARIMA Model
print("\n1. ARIMA Model:")
try:
    # Auto ARIMA with different orders
    arima_orders = [(1,0,1), (1,1,1), (2,1,2), (0,1,1)]
    arima_results = {}
    
    for order in arima_orders:
        try:
            model = ARIMA(y_ts, order=order)
            fitted_model = model.fit()
            arima_results[order] = {
                'model': fitted_model,
                'aic': fitted_model.aic,
                'bic': fitted_model.bic,
                'llf': fitted_model.llf
            }
            print(f"   ARIMA{order}: AIC={fitted_model.aic:.2f}, BIC={fitted_model.bic:.2f}")
        except Exception as e:
            print(f"   ARIMA{order}: Failed - {str(e)[:50]}...")
    
    # Select best ARIMA model
    if arima_results:
        best_order = min(arima_results.keys(), key=lambda x: arima_results[x]['aic'])
        best_arima = arima_results[best_order]['model']
        print(f"   ✅ Best ARIMA model: ARIMA{best_order}")
        
        # Generate forecast
        forecast_steps = 12  # 12 months ahead
        arima_forecast = best_arima.forecast(steps=forecast_steps)
        arima_conf_int = best_arima.get_forecast(steps=forecast_steps).conf_int()
        
        print(f"   📊 ARIMA Forecast (next 12 months):")
        print(f"      Mean: {arima_forecast.mean():.2f} million m³/month")
        print(f"      Range: {arima_forecast.min():.2f} - {arima_forecast.max():.2f}")
        
        # Calculate in-sample metrics
        fitted_values = best_arima.fittedvalues
        residuals = y_ts - fitted_values
        arima_mae = np.mean(np.abs(residuals))
        arima_rmse = np.sqrt(np.mean(residuals**2))
        arima_mape = np.mean(np.abs(residuals / y_ts)) * 100
        
        print(f"   📈 In-sample Performance:")
        print(f"      MAE: {arima_mae:.3f}")
        print(f"      RMSE: {arima_rmse:.3f}")
        print(f"      MAPE: {arima_mape:.2f}%")
        
except Exception as e:
    print(f"   ❌ ARIMA modeling failed: {str(e)}")
    best_arima = None

# 2. SARIMA Model (Seasonal ARIMA)
print("\n2. SARIMA Model:")
try:
    # Try different seasonal orders
    sarima_orders = [(1,0,1,12), (1,1,1,12), (2,1,2,12)]
    sarima_results = {}
    
    for order in sarima_orders:
        try:
            model = SARIMAX(y_ts, order=order[:3], seasonal_order=(1,1,1,12))
            fitted_model = model.fit(disp=False)
            sarima_results[order] = {
                'model': fitted_model,
                'aic': fitted_model.aic,
                'bic': fitted_model.bic
            }
            print(f"   SARIMA{order}: AIC={fitted_model.aic:.2f}, BIC={fitted_model.bic:.2f}")
        except Exception as e:
            print(f"   SARIMA{order}: Failed - {str(e)[:50]}...")
    
    # Select best SARIMA model
    if sarima_results:
        best_sarima_order = min(sarima_results.keys(), key=lambda x: sarima_results[x]['aic'])
        best_sarima = sarima_results[best_sarima_order]['model']
        print(f"   ✅ Best SARIMA model: SARIMA{best_sarima_order}")
        
        # Generate forecast
        sarima_forecast = best_sarima.forecast(steps=forecast_steps)
        sarima_conf_int = best_sarima.get_forecast(steps=forecast_steps).conf_int()
        
        print(f"   📊 SARIMA Forecast (next 12 months):")
        print(f"      Mean: {sarima_forecast.mean():.2f} million m³/month")
        print(f"      Range: {sarima_forecast.min():.2f} - {sarima_forecast.max():.2f}")
        
except Exception as e:
    print(f"   ❌ SARIMA modeling failed: {str(e)}")
    best_sarima = None

# 3. Exponential Smoothing
print("\n3. Exponential Smoothing (Holt-Winters):")
try:
    # Try different seasonal models
    exp_models = {}
    
    # Additive seasonal
    try:
        model_add = ExponentialSmoothing(y_ts, seasonal='add', seasonal_periods=12)
        fitted_add = model_add.fit()
        exp_models['additive'] = fitted_add
        print(f"   Additive Seasonal: AIC={fitted_add.aic:.2f}")
    except Exception as e:
        print(f"   Additive Seasonal: Failed - {str(e)[:50]}...")
    
    # Multiplicative seasonal
    try:
        model_mult = ExponentialSmoothing(y_ts, seasonal='mul', seasonal_periods=12)
        fitted_mult = model_mult.fit()
        exp_models['multiplicative'] = fitted_mult
        print(f"   Multiplicative Seasonal: AIC={fitted_mult.aic:.2f}")
    except Exception as e:
        print(f"   Multiplicative Seasonal: Failed - {str(e)[:50]}...")
    
    # Select best exponential smoothing model
    if exp_models:
        best_exp_type = min(exp_models.keys(), key=lambda x: exp_models[x].aic)
        best_exp = exp_models[best_exp_type]
        print(f"   ✅ Best Exponential Smoothing: {best_exp_type}")
        
        # Generate forecast
        exp_forecast = best_exp.forecast(steps=forecast_steps)
        
        print(f"   📊 Exponential Smoothing Forecast (next 12 months):")
        print(f"      Mean: {exp_forecast.mean():.2f} million m³/month")
        print(f"      Range: {exp_forecast.min():.2f} - {exp_forecast.max():.2f}")
        
except Exception as e:
    print(f"   ❌ Exponential Smoothing failed: {str(e)}")
    best_exp = None


In [ ]:
# === MACHINE LEARNING MODELS ===
print("\n🤖 MACHINE LEARNING FORECASTING")
print("=" * 50)

# Prepare data for ML models
print(f"ML dataset: X={X.shape}, y={y_aligned.shape}")

# Train-test split for ML models
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y_aligned.iloc[:split_idx], y_aligned.iloc[split_idx:]

print(f"Train set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ml_models = {}
ml_results = {}

# 1. Random Forest
print("\n1. Random Forest:")
try:
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    )
    rf_model.fit(X_train, y_train)
    
    # Predictions
    rf_pred_train = rf_model.predict(X_train)
    rf_pred_test = rf_model.predict(X_test)
    
    # Calculate metrics
    rf_mae_train = mean_absolute_error(y_train, rf_pred_train)
    rf_mae_test = mean_absolute_error(y_test, rf_pred_test)
    rf_rmse_train = np.sqrt(mean_squared_error(y_train, rf_pred_train))
    rf_rmse_test = np.sqrt(mean_squared_error(y_test, rf_pred_test))
    rf_r2_train = r2_score(y_train, rf_pred_train)
    rf_r2_test = r2_score(y_test, rf_pred_test)
    
    ml_models['Random Forest'] = rf_model
    ml_results['Random Forest'] = {
        'train_mae': rf_mae_train,
        'test_mae': rf_mae_test,
        'train_rmse': rf_rmse_train,
        'test_rmse': rf_rmse_test,
        'train_r2': rf_r2_train,
        'test_r2': rf_r2_test
    }
    
    print(f"   ✅ Random Forest trained successfully")
    print(f"   📊 Performance:")
    print(f"      Train - MAE: {rf_mae_train:.3f}, RMSE: {rf_rmse_train:.3f}, R²: {rf_r2_train:.3f}")
    print(f"      Test  - MAE: {rf_mae_test:.3f}, RMSE: {rf_rmse_test:.3f}, R²: {rf_r2_test:.3f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"   🔍 Top 5 Features:")
    for i, (_, row) in enumerate(feature_importance.head().iterrows(), 1):
        print(f"      {i}. {row['feature']:<25}: {row['importance']:.4f}")
        
except Exception as e:
    print(f"   ❌ Random Forest failed: {str(e)}")

# 2. XGBoost
print("\n2. XGBoost:")
try:
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    # Predictions
    xgb_pred_train = xgb_model.predict(X_train)
    xgb_pred_test = xgb_model.predict(X_test)
    
    # Calculate metrics
    xgb_mae_train = mean_absolute_error(y_train, xgb_pred_train)
    xgb_mae_test = mean_absolute_error(y_test, xgb_pred_test)
    xgb_rmse_train = np.sqrt(mean_squared_error(y_train, xgb_pred_train))
    xgb_rmse_test = np.sqrt(mean_squared_error(y_test, xgb_pred_test))
    xgb_r2_train = r2_score(y_train, xgb_pred_train)
    xgb_r2_test = r2_score(y_test, xgb_pred_test)
    
    ml_models['XGBoost'] = xgb_model
    ml_results['XGBoost'] = {
        'train_mae': xgb_mae_train,
        'test_mae': xgb_mae_test,
        'train_rmse': xgb_rmse_train,
        'test_rmse': xgb_rmse_test,
        'train_r2': xgb_r2_train,
        'test_r2': xgb_r2_test
    }
    
    print(f"   ✅ XGBoost trained successfully")
    print(f"   📊 Performance:")
    print(f"      Train - MAE: {xgb_mae_train:.3f}, RMSE: {xgb_rmse_train:.3f}, R²: {xgb_r2_train:.3f}")
    print(f"      Test  - MAE: {xgb_mae_test:.3f}, RMSE: {xgb_rmse_test:.3f}, R²: {xgb_r2_test:.3f}")
    
except Exception as e:
    print(f"   ❌ XGBoost failed: {str(e)}")

# 3. Support Vector Regression
print("\n3. Support Vector Regression:")
try:
    svr_model = SVR(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        epsilon=0.1
    )
    svr_model.fit(X_train_scaled, y_train)
    
    # Predictions
    svr_pred_train = svr_model.predict(X_train_scaled)
    svr_pred_test = svr_model.predict(X_test_scaled)
    
    # Calculate metrics
    svr_mae_train = mean_absolute_error(y_train, svr_pred_train)
    svr_mae_test = mean_absolute_error(y_test, svr_pred_test)
    svr_rmse_train = np.sqrt(mean_squared_error(y_train, svr_pred_train))
    svr_rmse_test = np.sqrt(mean_squared_error(y_test, svr_pred_test))
    svr_r2_train = r2_score(y_train, svr_pred_train)
    svr_r2_test = r2_score(y_test, svr_pred_test)
    
    ml_models['SVR'] = svr_model
    ml_results['SVR'] = {
        'train_mae': svr_mae_train,
        'test_mae': svr_mae_test,
        'train_rmse': svr_rmse_train,
        'test_rmse': svr_rmse_test,
        'train_r2': svr_r2_train,
        'test_r2': svr_r2_test
    }
    
    print(f"   ✅ SVR trained successfully")
    print(f"   📊 Performance:")
    print(f"      Train - MAE: {svr_mae_train:.3f}, RMSE: {svr_rmse_train:.3f}, R²: {svr_r2_train:.3f}")
    print(f"      Test  - MAE: {svr_mae_test:.3f}, RMSE: {svr_rmse_test:.3f}, R²: {svr_r2_test:.3f}")
    
except Exception as e:
    print(f"   ❌ SVR failed: {str(e)}")

# Model comparison
print(f"\n📊 MACHINE LEARNING MODEL COMPARISON:")
print("-" * 60)
print(f"{'Model':<15} {'Train MAE':<10} {'Test MAE':<10} {'Train R²':<10} {'Test R²':<10}")
print("-" * 60)

for model_name, results in ml_results.items():
    print(f"{model_name:<15} {results['train_mae']:<10.3f} {results['test_mae']:<10.3f} "
          f"{results['train_r2']:<10.3f} {results['test_r2']:<10.3f}")

# Find best model
if ml_results:
    best_ml_model = min(ml_results.keys(), key=lambda x: ml_results[x]['test_mae'])
    print(f"\n🏆 Best ML Model: {best_ml_model} (lowest test MAE)")


In [ ]:
# === PROPHET MODEL ===
print("\n🔮 PROPHET FORECASTING")
print("=" * 50)

try:
    # Prepare data for Prophet
    prophet_data = pd.DataFrame({
        'ds': gas_data.index,
        'y': gas_data[target_col]
    }).dropna()
    
    print(f"Prophet dataset: {len(prophet_data)} observations")
    
    # Initialize and fit Prophet model
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,  # Monthly data
        daily_seasonality=False,
        seasonality_mode='additive',
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    
    # Add custom seasonalities if needed
    prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    prophet_model.fit(prophet_data)
    
    # Create future dataframe for forecasting
    future_periods = 12  # 12 months ahead
    future = prophet_model.make_future_dataframe(periods=future_periods, freq='MS')
    
    # Generate forecast
    forecast = prophet_model.predict(future)
    
    # Extract forecast values
    prophet_forecast = forecast['yhat'].tail(future_periods)
    prophet_lower = forecast['yhat_lower'].tail(future_periods)
    prophet_upper = forecast['yhat_upper'].tail(future_periods)
    
    # Calculate in-sample metrics
    prophet_fitted = forecast['yhat'].iloc[:-future_periods]
    prophet_residuals = prophet_data['y'] - prophet_fitted
    prophet_mae = np.mean(np.abs(prophet_residuals))
    prophet_rmse = np.sqrt(np.mean(prophet_residuals**2))
    prophet_mape = np.mean(np.abs(prophet_residuals / prophet_data['y'])) * 100
    
    print(f"   ✅ Prophet model trained successfully")
    print(f"   📊 In-sample Performance:")
    print(f"      MAE: {prophet_mae:.3f}")
    print(f"      RMSE: {prophet_rmse:.3f}")
    print(f"      MAPE: {prophet_mape:.2f}%")
    print(f"   📈 Forecast (next 12 months):")
    print(f"      Mean: {prophet_forecast.mean():.2f} million m³/month")
    print(f"      Range: {prophet_forecast.min():.2f} - {prophet_forecast.max():.2f}")
    print(f"      Confidence interval: {prophet_lower.mean():.2f} - {prophet_upper.mean():.2f}")
    
except Exception as e:
    print(f"   ❌ Prophet modeling failed: {str(e)}")
    prophet_model = None


In [ ]:
# === ENSEMBLE FORECASTING ===
print("\n🎯 ENSEMBLE FORECASTING")
print("=" * 50)

# Collect all forecasts
all_forecasts = {}
forecast_dates = pd.date_range(start=gas_data.index.max() + pd.DateOffset(months=1), periods=12, freq='MS')

# Add time series forecasts
if 'best_arima' in locals() and best_arima is not None:
    all_forecasts['ARIMA'] = arima_forecast
if 'best_sarima' in locals() and best_sarima is not None:
    all_forecasts['SARIMA'] = sarima_forecast
if 'best_exp' in locals() and best_exp is not None:
    all_forecasts['Exponential Smoothing'] = exp_forecast
if 'prophet_model' in locals() and prophet_model is not None:
    all_forecasts['Prophet'] = prophet_forecast.values

# Generate ML forecasts for future periods
print("Generating ML forecasts for future periods...")

# Create future feature matrix for ML models
last_date = X.index.max()
future_features = []

for i, future_date in enumerate(forecast_dates):
    feature_row = {}
    
    # Use seasonal patterns for future features
    for col in X.columns:
        if 'month_sin' in col or 'month_cos' in col:
            month = future_date.month
            if 'sin' in col:
                feature_row[col] = np.sin(2 * np.pi * month / 12)
            else:
                feature_row[col] = np.cos(2 * np.pi * month / 12)
        elif 'day_of_year_sin' in col or 'day_of_year_cos' in col:
            day_of_year = future_date.timetuple().tm_yday
            if 'sin' in col:
                feature_row[col] = np.sin(2 * np.pi * day_of_year / 365.25)
            else:
                feature_row[col] = np.cos(2 * np.pi * day_of_year / 365.25)
        elif 'is_winter' in col:
            feature_row[col] = 1 if future_date.month in [12, 1, 2] else 0
        elif 'is_spring' in col:
            feature_row[col] = 1 if future_date.month in [3, 4, 5] else 0
        elif 'is_summer' in col:
            feature_row[col] = 1 if future_date.month in [6, 7, 8] else 0
        elif 'is_autumn' in col:
            feature_row[col] = 1 if future_date.month in [9, 10, 11] else 0
        elif 'avg_temperature_celsius' in col:
            # Seasonal temperature estimate
            seasonal_temp = 15 + 10 * np.sin(2 * np.pi * (future_date.month - 7) / 12)
            feature_row[col] = seasonal_temp
        elif 'heating_degree_days' in col:
            temp = 15 + 10 * np.sin(2 * np.pi * (future_date.month - 7) / 12)
            feature_row[col] = max(0, 18 - temp) * 30
        elif 'cooling_degree_days' in col:
            temp = 15 + 10 * np.sin(2 * np.pi * (future_date.month - 7) / 12)
            feature_row[col] = max(0, temp - 24) * 30
        elif 'lag_' in col or 'ma_' in col or 'std_' in col:
            # Use historical averages for lag features
            feature_row[col] = X[col].mean()
        else:
            # Use historical mean for other features
            feature_row[col] = X[col].mean()
    
    future_features.append(feature_row)

future_X = pd.DataFrame(future_features)

# Generate ML forecasts
for model_name, model in ml_models.items():
    try:
        if model_name == 'SVR':
            future_X_scaled = scaler.transform(future_X)
            ml_forecast = model.predict(future_X_scaled)
        else:
            ml_forecast = model.predict(future_X)
        
        all_forecasts[model_name] = ml_forecast
        print(f"   ✅ {model_name} forecast generated")
        
    except Exception as e:
        print(f"   ❌ {model_name} forecast failed: {str(e)}")

# Create ensemble forecast
if len(all_forecasts) > 1:
    print(f"\n🎯 Creating ensemble forecast from {len(all_forecasts)} models...")
    
    # Simple average ensemble
    ensemble_values = np.mean(list(all_forecasts.values()), axis=0)
    
    # Weighted ensemble (based on inverse MAE)
    weights = {}
    for model_name in all_forecasts.keys():
        if model_name in ml_results:
            weights[model_name] = 1 / ml_results[model_name]['test_mae']
        else:
            weights[model_name] = 1.0  # Equal weight for time series models
    
    # Normalize weights
    total_weight = sum(weights.values())
    weights = {k: v/total_weight for k, v in weights.items()}
    
    weighted_ensemble = np.zeros(len(forecast_dates))
    for model_name, forecast in all_forecasts.items():
        weighted_ensemble += weights[model_name] * forecast
    
    print(f"   📊 Ensemble Forecast Results:")
    print(f"      Simple Average: {ensemble_values.mean():.2f} ± {ensemble_values.std():.2f} million m³/month")
    print(f"      Weighted Average: {weighted_ensemble.mean():.2f} ± {weighted_ensemble.std():.2f} million m³/month")
    
    # Store ensemble forecasts
    all_forecasts['Ensemble (Simple)'] = ensemble_values
    all_forecasts['Ensemble (Weighted)'] = weighted_ensemble
    
else:
    print("   ⚠️ Insufficient models for ensemble forecasting")

print(f"\n📈 FORECAST SUMMARY:")
print("-" * 60)
print(f"{'Model':<20} {'Mean Forecast':<15} {'Std Dev':<10} {'Range':<20}")
print("-" * 60)

for model_name, forecast in all_forecasts.items():
    print(f"{model_name:<20} {forecast.mean():<15.2f} {forecast.std():<10.2f} "
          f"{forecast.min():.1f}-{forecast.max():.1f}")

print(f"\n✅ All forecasting models completed successfully!")


In [ ]:
# === FORECAST VISUALIZATION ===
print("\n📊 CREATING FORECAST VISUALIZATION")
print("=" * 50)

# Create comprehensive forecast visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Japan Gas Demand Forecasting - Model Comparison', fontsize=16, fontweight='bold')

# 1. Historical data with forecasts
ax1 = axes[0, 0]
ax1.plot(gas_data.index, gas_data[target_col], 'b-', linewidth=2, label='Historical Data', alpha=0.8)

# Plot individual forecasts
colors = ['red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']
for i, (model_name, forecast) in enumerate(all_forecasts.items()):
    if len(forecast) == len(forecast_dates):
        ax1.plot(forecast_dates, forecast, '--', color=colors[i % len(colors)], 
                linewidth=2, alpha=0.7, label=f'{model_name}')

ax1.set_title('Historical Data vs Model Forecasts', fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Gas Demand (Million m³/month)')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. Forecast comparison (bar chart)
ax2 = axes[0, 1]
model_names = list(all_forecasts.keys())
forecast_means = [forecast.mean() for forecast in all_forecasts.values()]
forecast_stds = [forecast.std() for forecast in all_forecasts.values()]

bars = ax2.bar(range(len(model_names)), forecast_means, yerr=forecast_stds, 
               capsize=5, alpha=0.7, color=colors[:len(model_names)])
ax2.set_title('Forecast Comparison (12-Month Average)', fontweight='bold')
ax2.set_xlabel('Model')
ax2.set_ylabel('Average Forecast (Million m³/month)')
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, mean_val in zip(bars, forecast_means):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
             f'{mean_val:.1f}', ha='center', va='bottom', fontweight='bold')

# 3. Model performance comparison (if available)
ax3 = axes[1, 0]
if ml_results:
    ml_names = list(ml_results.keys())
    test_maes = [ml_results[name]['test_mae'] for name in ml_names]
    
    bars = ax3.bar(ml_names, test_maes, alpha=0.7, color=['red', 'green', 'blue'][:len(ml_names)])
    ax3.set_title('Model Performance (Test MAE)', fontweight='bold')
    ax3.set_ylabel('Test MAE (Million m³/month)')
    ax3.set_xlabel('Model')
    
    # Add value labels
    for bar, mae in zip(bars, test_maes):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{mae:.3f}', ha='center', va='bottom', fontweight='bold')
    
    ax3.grid(True, alpha=0.3, axis='y')
else:
    ax3.text(0.5, 0.5, 'ML Performance\nData Not Available', 
             ha='center', va='center', transform=ax3.transAxes, fontsize=12)
    ax3.set_title('Model Performance - No Data', fontweight='bold')

# 4. Forecast uncertainty (if ensemble available)
ax4 = axes[1, 1]
if 'Ensemble (Simple)' in all_forecasts:
    ensemble_forecast = all_forecasts['Ensemble (Simple)']
    
    # Calculate confidence intervals
    forecast_mean = ensemble_forecast.mean()
    forecast_std = ensemble_forecast.std()
    
    # Plot forecast with confidence bands
    ax4.plot(forecast_dates, ensemble_forecast, 'b-', linewidth=2, label='Ensemble Forecast')
    ax4.fill_between(forecast_dates, 
                     ensemble_forecast - 1.96 * forecast_std,
                     ensemble_forecast + 1.96 * forecast_std,
                     alpha=0.3, color='blue', label='95% Confidence Interval')
    
    ax4.set_title('Ensemble Forecast with Uncertainty', fontweight='bold')
    ax4.set_xlabel('Date')
    ax4.set_ylabel('Gas Demand (Million m³/month)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Ensemble Forecast\nNot Available', 
             ha='center', va='center', transform=ax4.transAxes, fontsize=12)
    ax4.set_title('Ensemble Forecast - No Data', fontweight='bold')

plt.tight_layout()
plt.show()

# Create detailed forecast table
print(f"\n📋 DETAILED FORECAST TABLE:")
print("=" * 80)

# Create forecast DataFrame
forecast_df = pd.DataFrame({
    'Date': forecast_dates,
    **{model_name: forecast for model_name, forecast in all_forecasts.items()}
})

print("Next 12 months forecast (Million m³/month):")
print(forecast_df.round(2))

# Summary statistics
print(f"\n📊 FORECAST SUMMARY STATISTICS:")
print("-" * 50)
print(f"• Total models trained: {len(all_forecasts)}")
print(f"• Forecast horizon: 12 months")
print(f"• Forecast period: {forecast_dates[0].strftime('%Y-%m')} to {forecast_dates[-1].strftime('%Y-%m')}")

if 'Ensemble (Simple)' in all_forecasts:
    ensemble_mean = all_forecasts['Ensemble (Simple)'].mean()
    ensemble_std = all_forecasts['Ensemble (Simple)'].std()
    print(f"• Recommended forecast (Ensemble): {ensemble_mean:.2f} ± {ensemble_std:.2f} million m³/month")
    print(f"• Annual forecast: {ensemble_mean * 12:.0f} million m³/year")

print(f"\n✅ Forecast visualization and analysis complete!")


# Japan Gas Demand Forecasting Models

This notebook implements and compares multiple forecasting approaches for short-term gas demand in Japan, including traditional time series models and machine learning techniques.

## Objectives
1. Implement ARIMA/SARIMA models for time series forecasting
2. Build exponential smoothing models (Holt-Winters)
3. Develop machine learning models (Random Forest, XGBoost)
4. Create ensemble forecasting approaches
5. Compare model performance using appropriate metrics
6. Generate short-term forecasts with uncertainty estimates

In [ ]:
# Import required libraries for forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta

# Time series modeling
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Evaluation metrics
from sklearn.metrics import mean_absolute_percentage_error

# Plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set styling and suppress warnings
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')
np.random.seed(42)

print("🚀 Forecasting libraries imported successfully!")
print(f"Model development started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Recreate the synthetic gas demand data for modeling
np.random.seed(42)

# Create date range and synthetic data (same as EDA notebook)
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2024-08-31') 
dates = pd.date_range(start_date, end_date, freq='MS')

def generate_realistic_gas_data():
    """Generate synthetic but realistic Japanese gas demand data"""
    n_months = len(dates)
    base_monthly = 37000 / 12  # Million cubic meters per month
    
    # Components
    trend = np.linspace(1.0, 0.95, n_months)
    months = np.array([d.month for d in dates])
    seasonal = 1.0 + 0.4 * np.cos(2 * np.pi * (months - 1) / 12)
    
    # COVID impact
    covid_impact = np.ones(n_months)
    for i, date in enumerate(dates):
        if pd.Timestamp('2020-03-01') <= date <= pd.Timestamp('2021-06-01'):
            covid_impact[i] = 0.85
    
    noise = 1.0 + np.random.normal(0, 0.05, n_months)
    gas_demand = base_monthly * trend * seasonal * covid_impact * noise
    
    # Weather variables
    avg_temp = np.array([15 + 10 * np.cos(2 * np.pi * (d.month - 7) / 12) + 
                        np.random.normal(0, 2) for d in dates])
    hdd = np.maximum(0, 18 - avg_temp) * 30
    cdd = np.maximum(0, avg_temp - 22) * 30
    
    # Economic variables
    gdp_growth = np.random.normal(0.8, 1.0, n_months)
    gas_price = 45 + 10 * np.sin(2 * np.pi * np.arange(n_months) / 12) + np.random.normal(0, 2, n_months)
    
    return pd.DataFrame({
        'date': dates,
        'gas_demand': gas_demand,
        'temperature': avg_temp,
        'heating_degree_days': hdd,
        'cooling_degree_days': cdd,
        'gdp_growth': gdp_growth,
        'gas_price': gas_price
    }).set_index('date')

# Generate the dataset
gas_data = generate_realistic_gas_data()

print("📊 DATASET FOR FORECASTING MODELS")
print("=" * 50)
print(f"Period: {gas_data.index.min().strftime('%Y-%m')} to {gas_data.index.max().strftime('%Y-%m')}")
print(f"Observations: {len(gas_data)} monthly records")
print(f"Variables: {len(gas_data.columns)}")
print(f"\nTarget variable statistics:")
print(f"Mean gas demand: {gas_data['gas_demand'].mean():.1f} million m³/month")
print(f"Standard deviation: {gas_data['gas_demand'].std():.1f} million m³/month")
print(f"Coefficient of variation: {(gas_data['gas_demand'].std()/gas_data['gas_demand'].mean()*100):.1f}%")

# Display first and last few observations
print(f"\nFirst 5 observations:")
print(gas_data.head())
print(f"\nLast 5 observations:")
print(gas_data.tail())

## 1. Model Evaluation Framework

Setting up comprehensive evaluation metrics and validation procedures for time series forecasting.